In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

In [2]:
# Import data as pandas dataframe
df = pd.read_csv("..\Data\creditcard.csv")

# Convert Class to category (0: Legit | 1: Fraud)
df.Class = df.Class.astype('category')
print(df.shape)
df.head(3)

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [3]:
# Separates the attributes (X) from the target (y)
# Uses the top 10 attributes that contribute to y and drops the rest (Ian's research)
X = df[['V17', 'V14', 'V12', 'V10', 'V11', 'V16', 'V4', 'V3', 'V18', 'V9']]
y = df.Class
print(X.shape)
X.head(3)

(284807, 10)


,V17,V14,V12,V10,V11,V16,V4,V3,V18,V9
0,0.207971,-0.311169,-0.617801,0.090794,-0.551600,-0.470401,1.378155,2.536347,0.025791,0.363787
1,-0.114805,-0.143772,1.065235,-0.166974,1.612727,0.463917,0.448154,0.166480,-0.183361,-0.255425
2,1.109969,-0.165946,0.066084,0.207643,0.624501,-2.890083,0.379780,1.773209,-0.121359,-1.514654


In [4]:
# Do Train/Test Split with 80% of data as training set and the remaing 20% as the test set
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.20)

In [5]:
# Set up for keeping track of scores
K = []
Precision = []
Recall = []
F1 = []
AUPRC = []

# Gets scores for SMOTE with 1 to 100 neightbors
for k in range(100):
    # Perform SMOTE on data using k+1 neighbours (From 1 to 100)
    sm = SMOTE(random_state=42, k_neighbors=k+1)
    X_res,y_res = sm.fit_resample(X_train, y_train) 
    
    # Runs Gaussian Naive Bayes on SMOTE upsampled data
    nb = GaussianNB()
    nb.fit(X_res, y_res)
    
    # Predicts y using the test set's attributes
    y_h = nb.predict(X_test)
    
    # Places scores in arrays
    K.append(k+1)
    Precision.append(precision_score(y_test, y_h))
    Recall.append(recall_score(y_test, y_h))
    F1.append(f1_score(y_test, y_h))
    AUPRC.append(average_precision_score(y_true=y_test, y_score=y_h))

In [6]:
# Puts all scores into a data frame to be viewed later
results = pd.DataFrame()
results['K'] = K
results['Precision'] = Precision
results['Recall'] = Recall
results['F1'] = F1
results['AUPRC'] = AUPRC
results.head(3)

,K,Precision,Recall,F1,AUPRC
0,1,0.112172,0.817391,0.197272,0.092057
1,2,0.112845,0.817391,0.198312,0.092607
2,3,0.112575,0.817391,0.197895,0.092386


In [7]:
# Finds the index with the largest  AUPRC value and displays it
max_idx = results['AUPRC'].idxmax()
results.iloc[max_idx:(max_idx+1)].round(3)

,K,Precision,Recall,F1,AUPRC
96,97,0.118,0.826,0.207,0.098
